In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
train_df = pd.read_csv('../data/raw/train.csv')
train_df.sample(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
106,107,1,3,"Salkjelsvik, Miss. Anna Kristine",female,21.0,0,0,343120,7.6500,NaN,S
23,24,1,1,"Sloper, Mr. William Thompson",male,28.0,0,0,113788,35.5000,A6,S
862,863,1,1,"Swift, Mrs. Frederick Joel (Margaret Welles Ba...",female,48.0,0,0,17466,25.9292,D17,S
415,416,0,3,"Meek, Mrs. Thomas (Annie Louise Rowley)",female,NaN,0,0,343095,8.0500,NaN,S
448,449,1,3,"Baclini, Miss. Marie Catherine",female,5.0,2,1,2666,19.2583,NaN,C


In [3]:
test_df = pd.read_csv('../data/raw/test.csv')
# test_df.sample(10)

In [4]:
def xtract(df):
    cpy = df.copy()
      
    cpy = pd.concat([df,pd.get_dummies(df["Sex"],prefix='sex')],axis=1)
    cpy = pd.concat([cpy,pd.get_dummies(df["Pclass"],prefix='class')],axis=1)
    
    # only columns we'll actually use
    cpy = cpy[ [col for col in list(cpy) if col.startswith('sex')  or col.startswith('class') or col == "Age" or col == "Survived" ] ]
       
    return cpy.dropna()
        
train_df_clean = xtract(train_df)

In [5]:
test_df_clean = xtract(test_df)

In [6]:
X = []
y = []

for row in train_df_clean.values:
    X.append(row[1:])
    y.append(row[0])
    
X = np.array(X)
y = np.array(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

In [7]:
# comment this block to see the difference
scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [8]:
# purposefully ignoring data standardization to mean 0 and variance 1
clf = MLPClassifier(random_state=1)
clf.fit(X_train,y_train)
metrics.accuracy_score(y_test,clf.predict(X_test))

0.83216783216783219